# Pathology prediction (Cardiomegaly)



@References : Soenksen, L.R., Ma, Y., Zeng, C. et al. Integrated multimodal artificial intelligence framework for healthcare applications. npj Digit. Med. 5, 149 (2022). https://doi.org/10.1038/s41746-022-00689-4

In this notebook, the task is to predict the Cardiomegaly pathology using the CSV embeddings file

## Introduction


Radiology notes were processed to determine if each of the pathologies was explicitly confirmed as present (value = 1), explicitly confirmed as absent (value = 0), inconclusive in the study (value = −1), or not explored (no value).

Selected samples : 0 or 1 values (removing the rest from the training and testing data).

Excluded variables : the unstructured radiology notes component (E_rad ) from the allowable input to avoid potential overfitting or misrepresentations of real predictive value.

The model is based on binary classification for each target chest pathology.

Final sample size for the Cardiomegaly pathology is : (N = 18,571)

#### Imports

In [1]:
import os
os.chdir('../')

from pandas import read_csv

from src.data import constants
from src.data.dataset import HAIMDataset
from src.evaluation.pycaret_evaluator import PyCaretEvaluator
from src.utils.metric_scores import *

#### Read data from local source



In [2]:
df = read_csv(constants.FILE_DF, nrows=constants.N_DATA)

#### Create a custom dataset for the HAIM experiment


Build the target column for the task at hand, set the dataset specificities:  the ``haim_id`` as a ``global_id``, use all sources except ``radiology notes`` 

In [3]:
dataset = HAIMDataset(df,  
                      constants.CHEST_PREDICTORS, 
                      constants.ALL_MODALITIES, 
                      constants.CARDIOMEGALY, 
                      constants.IMG_ID, 
                      constants.GLOBAL_ID)

#### Set hyper-parameters

In [4]:
# Define the grid oh hyper-parameters for the tuning
grid_hps = {'max_depth': [5, 6, 7, 8],
            'n_estimators': [200, 300],
            'learning_rate': [0.3, 0.1, 0.05],
            }

### Model training and predictions using an XGBClassifier model with GridSearchCV and Hyperparameters optimization


The goal of this section of the notebook is to compute the following metrics:

``ACCURACY_SCORE, BALANCED_ACCURACY_SCORE, SENSITIVITY, SPECIFICITY, AUC, BRIER SCORE, BINARY CROSS-ENTROPY``


The
hyperparameter combinations of individual XGBoost models were
selected within each training loop using a ``fivefold cross-validated
grid search`` on the training set (80%). This XGBoost ``tuning process``
selected the ``maximum depth of the trees (5–8)``, the number of
``estimators (200 or 300)``, and the ``learning rate (0.05, 0.1, 0.3)``
according to the parameter value combination leading to the
highest observed AUROC within the training loop 


As mentioned previously, all XGBoost models were trained ``five times with five different data splits`` to repeat the
experiments and compute average metrics 


```Refer to page 8 of study``` : https://doi.org/10.1038/s41746-022-00689-4

In [5]:
# Initialize the PyCaret Evaluator
evaluator = PyCaretEvaluator(dataset=dataset, target="Cardiomegaly", experiment_name="CP_Cardiomegaly", filepath="./results/cardiomegaly")

# Model training
evaluator.run_experiment(
    train_size=0.8,
    fold=5,
    fold_strategy='kfold',
    outer_fold=5,
    outer_strategy='kfold',
    session_id=42,
    model='xgboost',
    optimize='AUC',
    custom_grid=grid_hps
)

2024-10-07 11:30:51,125	INFO worker.py:1777 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


(run_fold pid=1153833) Outer fold 1
(run_fold pid=1153847) Configuring PyCaret for outer fold 2
(run_fold pid=1153841) Outer fold 3 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


Processing:  75%|███████▌  | 3/4 [07:48<02:53, 173.45s/it] [repeated 2x across cluster]


(run_fold pid=1153841)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=1153841) Fold                                                          
(run_fold pid=1153841) 0       0.9053  0.8972  0.9783  0.9166  0.9464  0.5433  0.5656
(run_fold pid=1153841) 1       0.9146  0.9262  0.9812  0.9232  0.9513  0.6057  0.6251
(run_fold pid=1153841) 2       0.9037  0.9075  0.9770  0.9145  0.9448  0.5719  0.5916
(run_fold pid=1153841) 3       0.9028  0.9023  0.9736  0.9166  0.9442  0.5688  0.5850
(run_fold pid=1153841) 4       0.9087  0.9167  0.9749  0.9225  0.9480  0.5781  0.5931
(run_fold pid=1153841) Mean    0.9070  0.9100  0.9770  0.9187  0.9469  0.5736  0.5921
(run_fold pid=1153841) Std     0.0043  0.0104  0.0027  0.0035  0.0026  0.0200  0.0192
(run_fold pid=1153841) Configuring PyCaret for outer fold 3 [repeated 2x across cluster]
(run_fold pid=1153841) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=1153833) 0       0.9121  0.9071  0.9783  0.9233  0.9500  0.5878  0.6053
(run_fold pid=1153833) 1       0.9083  0.9125  0.9768  0.9203  0.9477  0.5783  0.5957
(run_fold pid=1153833) 2       0.9095  0.9247  0.9820  0.9166  0.9482  0.5950  0.6178
(run_fold pid=1153833) 3       0.8990  0.9204  0.9701  0.9154  0.9419  0.5573  0.5715
(run_fold pid=1153833) 4       0.9082  0.8986  0.9773  0.9200  0.9478  0.5721  0.5905
(run_fold pid=1153833)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=1153833) Fold                                                          
(run_fold pid=1153833) Mean    0.9074  0.9127  0.9769  0.9191  0.9471  0.5781  0.5962
(run_fold pid=1153833) Std     0.0044  0.0093  0.0039  0.0028  0.0027  0.0130  0.0154
(run_fold pid=1153833) Tuning hyperparameters for model xgboost with custom grid using grid search
(run_fold pid=1153847)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=1153847) Fold              

...

(run_fold pid=1153833) Transformation Pipeline and Model Successfully Saved
(run_fold pid=1153833)                        Model  Accuracy     AUC  ...      F1   Kappa     MCC
(run_fold pid=1153833) 0  Extreme Gradient Boosting    0.9133  0.9293  ...  0.9501  0.6221  0.6411
(run_fold pid=1153833) 
(run_fold pid=1153833) [1 rows x 8 columns]


Processing:  75%|███████▌  | 3/4 [07:26<02:45, 165.17s/it]


(run_fold pid=1153841)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=1153841) Fold                                                          
(run_fold pid=1153841) 0       0.9095  0.9161  0.9762  0.9220  0.9483  0.5881  0.6040
(run_fold pid=1153841) 1       0.8986  0.9058  0.9688  0.9165  0.9419  0.5460  0.5592
(run_fold pid=1153841) 2       0.9032  0.9066  0.9794  0.9115  0.9442  0.5838  0.6059
(run_fold pid=1153841) 3       0.9045  0.9082  0.9736  0.9181  0.9450  0.5842  0.5991
(run_fold pid=1153841) 4       0.9074  0.9088  0.9792  0.9175  0.9474  0.5674  0.5889
(run_fold pid=1153841) Mean    0.9047  0.9091  0.9754  0.9171  0.9454  0.5739  0.5914
(run_fold pid=1153841) Std     0.0037  0.0037  0.0040  0.0034  0.0023  0.0157  0.0171
(run_fold pid=1153841) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=1153847) 0       0.9184  0.9212  0.9783  0.9297  0.9534  0.6285  0.6418
(run_fold pid=1153847) 1       0.9074  0.9139  0.9787  0.9178  0.9472  0.5737  0.5941
(run_fold pid=1153847) 2       0.9049  0.9130  0.9775  0.9153  0.9454  0.5827  0.6020
(run_fold pid=1153847) 3       0.9003  0.9176  0.9751  0.9127  0.9429  0.5557  0.5750
(run_fold pid=1153847) 4       0.9045  0.9124  0.9783  0.9154  0.9458  0.5505  0.5726
(run_fold pid=1153847)       Accuracy     AUC  Recall   Prec.      F1   Kappa     MCC
(run_fold pid=1153847) Fold                                                          
(run_fold pid=1153847) Mean    0.9071  0.9156  0.9776  0.9181  0.9469  0.5782  0.5971
(run_fold pid=1153847) Std     0.0061  0.0033  0.0013  0.0060  0.0035  0.0277  0.0250
(run_fold pid=1153847) Tuning hyperparameters for model xgboost with custom grid using grid search


(run_fold pid=1153841) Transformation Pipeline and Model Successfully Saved
(run_fold pid=1153841)                        Model  Accuracy    AUC  ...      F1  Kappa     MCC
(run_fold pid=1153841) 0  Extreme Gradient Boosting    0.9179  0.938  ...  0.9532  0.619  0.6351
(run_fold pid=1153841) 
(run_fold pid=1153841) [1 rows x 8 columns]
(run_fold pid=1153847) Transformation Pipeline and Model Successfully Saved
(run_fold pid=1153847)                        Model  Accuracy    AUC  ...      F1   Kappa     MCC
(run_fold pid=1153847) 0  Extreme Gradient Boosting    0.9122  0.924  ...  0.9499  0.5981  0.6219
(run_fold pid=1153847) 
(run_fold pid=1153847) [1 rows x 8 columns]
Final metrics table:
     Metric     Mean   Std Dev
0  Accuracy  0.91284  0.003278
1       AUC  0.92190  0.006277
2    Recall  0.98116  0.002048
3     Prec.  0.92166  0.004258
4        F1  0.95044  0.001777
5     Kappa  0.59052  0.021848
6       MCC  0.61154  0.017740
